In [6]:
#Data management
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import cv2
import re
import string
import os
from tqdm import tqdm

#Model management
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import torch
import joblib

#data visualize
import matplotlib.pyplot as plt
import seaborn as sns

#image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [7]:
### CHANGE from 
#df = pd.read_csv("../input/w1n1p2-one/df.csv")
#img_folder = "../input/w1n1p2-one/crop_images"
### original file is w1n1p2-one    to w1w1-data-v2

In [10]:
df = pd.read_csv("../input/w1n1-data-v2/df.csv")
img_folder = "../input/w1n1-data-v2/crop_images"
y_col = "fontWeight"
unique_y = len(df[y_col].unique())
target_size = (25,200)  ##previous data already set to 25,200
df.head()

In [12]:
df[y_col].unique()

In [13]:
df.shape

In [14]:
from sklearn.model_selection import train_test_split
#_ , df = train_test_split(df,train_size=0.4, random_state=5,stratify = df["fontFamily"])
df_vali , df_train = train_test_split(df,train_size=0.1, random_state=5,stratify = df[y_col])
print(df_train.shape[0])

In [17]:
### x_col="file"  = name of the image
### target = y_col
#rescale=1./255.  -->  0,255 ->  0,1
# batch_size = more the faster

In [16]:
datagen = ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory=img_folder,
x_col="file",
y_col=y_col,
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=target_size)

valid_generator=datagen.flow_from_dataframe(
dataframe=df_vali,
directory=img_folder,
x_col="file",
y_col=y_col,
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=target_size)

In [19]:
index = range(len(train_generator.class_indices.keys()))
fonts = [[x] for x in list(train_generator.class_indices.keys())]
decode = dict(zip(index, fonts))
decode = pd.DataFrame.from_dict(decode)
decode.index = [y_col]
decode.to_csv("decode.csv")
decode

In [20]:
### shape = (*target_size,3)   -->   (25,200,3)

In [ ]:
# ### convo normal
# def CRNN(shape = (*target_size,3)):
#     inputs = layers.Input(shape=shape)

#     conv_1 = layers.Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
#     pool_1 = layers.MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)

#     conv_2 = layers.Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
#     pool_2 = layers.MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

#     conv_3 = layers.Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)

#     conv_4 = layers.Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
#     pool_4 = layers.MaxPool2D(pool_size=(2, 1))(conv_4)

#     conv_5 = layers.Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
#     batch_norm_5 = layers.BatchNormalization()(conv_5)

#     conv_6 = layers.Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
#     batch_norm_6 = layers.BatchNormalization()(conv_6)
#     pool_6 = layers.MaxPool2D(pool_size=(2, 1))(batch_norm_6)

#     conv_7 = layers.Conv2D(512, (2,2), activation = 'relu', padding='same')(pool_6)
    
#     x = layers.Flatten()(conv_7)
#     x = layers.Dense(1024, activation = 'relu')(x)
#     x = layers.Dropout(0.3)(x)
#     x = layers.Dense(1024, activation = 'relu')(x)
#     x = layers.Dropout(0.3)(x)
#     outputs = layers.Dense(unique_y, activation = 'softmax')(x)

#     act_model = Model(inputs, outputs)
#     act_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
#     act_model.summary()
#     return act_model

In [22]:
def CRNN(shape = (*target_size,3)):
    inputs = layers.Input(shape=shape)

    conv_1 = layers.Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
    pool_1 = layers.MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)

    conv_2 = layers.Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
    pool_2 = layers.MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

    conv_3 = layers.Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)

    conv_4 = layers.Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
    pool_4 = layers.MaxPool2D(pool_size=(2, 1))(conv_4)

    conv_5 = layers.Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
    batch_norm_5 = layers.BatchNormalization()(conv_5)

    conv_6 = layers.Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
    batch_norm_6 = layers.BatchNormalization()(conv_6)
    pool_6 = layers.MaxPool2D(pool_size=(2, 1))(batch_norm_6)

    conv_7 = layers.Conv2D(512, (2,2), activation = 'relu', padding='same')(pool_6)
    
    ### LSTM good example###

    squeezed = layers.Lambda(lambda x: tf.squeeze(x, 1))(conv_7)

    blstm_1 = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
    blstm_2 = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)
    
    ### LSTM END###
    x = layers.Flatten()(blstm_2)
    x = layers.Dense(1024, activation = 'relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(1024, activation = 'relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(unique_y, activation = 'softmax')(x)

    act_model = Model(inputs, outputs)
    act_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    act_model.summary()
    return act_model

In [23]:
model = CRNN()

In [24]:
### train data to change the weight -> prevent overfit### patience = number of epoch to overfit
ES = EarlyStopping(monitor='val_loss', patience=5, verbose=1,min_delta=0,restore_best_weights=True)
model.fit(train_generator,validation_data=valid_generator,epochs=100,callbacks = [ES])

In [ ]:
### time
### gpu =14mins

In [25]:
try:
    os.mkdir("./model")
except:
    pass
model.save('./model_weight')